In [1]:
# Check platform.
import platform
if platform.machine() not in ['x86_64', 'aarch64']:
    raise SystemExit("Unsupported platform!")

import math
import os
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the channel estimator and some utilities for converting
# the DMRS fields in the right format from the SCF FAPI format that the dataset follows.
from aerial.phy5g.algorithms import ChannelEstimator
from aerial.util.fapi import dmrs_fapi_to_bit_array

import torch
import torch.nn as nn

# !pip install torchinfo
from torchinfo import summary

# Connecting to clickhouse on remote server
import clickhouse_connect
clickhouse_client = clickhouse_connect.get_client(host='localhost')

In [2]:
print(torch.cuda.device_count())  # Number of GPUs available
device = torch.device("cuda:0")  # Select the first GPU (index 0)
print(device)

1
cuda:0


In [3]:
import dill
model = torch.load("model_w10_i4_o1_h50_l1.pth", pickle_module=dill)

In [4]:
WINDOWS_SIZE = 10

In [5]:
model_stats = summary(model, input_size=(1, WINDOWS_SIZE, 4))
print(model_stats)

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     [1]                       --
├─LSTM: 1-1                              [1, 10, 50]               11,200
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 1]                    51
Total params: 11,251
Trainable params: 11,251
Non-trainable params: 0
Total mult-adds (M): 0.11
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05


In [7]:
while True:
    query = f"""
    SELECT * FROM MAC_KPIs 
    ORDER BY TsTaiNs DESC
    LIMIT {WINDOWS_SIZE}
    """
    # print(query)
    kpis = clickhouse_client.query_df(query)
    # print(kpis)

    x = kpis[['phr', 'wb_cqi', 'pusch_snr', 'rsrp']].to_numpy() / np.array([100, 20, 30, -150])
    timestamp = kpis['TsTaiNs'].iloc[-1]

    X_torch = torch.tensor([x], dtype=torch.float32)
    predicted = model(X_torch.to(device)).to("cpu").detach().numpy()
    print(f" {timestamp}: {predicted[-1]}")
    time.sleep(1)

 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047
 2025-04-11 10:19:26.412000: 37.44750213623047


KeyboardInterrupt: 